In [ ]:
import glob, sys, os
from elasticsearch import Elasticsearch
from requests.auth import HTTPBasicAuth
from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer, models
import ast

In [ ]:
import requests
import pandas as pd
import itertools

from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

In [ ]:
load_dotenv()
project_id = os.environ["PROJECT_ID"]
ibm_cloud_url = os.environ["IBM_CLOUD_URL"]
api_key = os.environ["API_KEY"]

In [ ]:
load_dotenv()
watsonx_discovery_username=os.environ["WATSONX_DISCOVERY_USERNAME"]
watsonx_discovery_password=os.environ["WATSONX_DISCOVERY_PASSWORD"]
watsonx_discovery_url=os.environ["WATSONX_DISCOVERY_URL"]
watsonx_discovery_port=os.environ["WATSONX_DISCOVERY_PORT"]
watsonx_discovery_endpoint = watsonx_discovery_url+':'+watsonx_discovery_port

# Connect to Elasticsearch with basic authentication
es = Elasticsearch(
    [watsonx_discovery_endpoint],
    http_auth=(watsonx_discovery_username, watsonx_discovery_password),
    verify_certs=False
)

# Checking if the connection was successful
if es.ping():
    print("Connection to Elasticsearch successful")
else:
    print("Connection to Elasticsearch failed")

# Dynamic Prompt Preparation

In [ ]:
table1 = """
Table1: SALES_TRANSACTION 

Table Description: This table stores information about individual sales transactions, including the transaction details, customer information, and sales amounts. 
Table Type: Fact table 
Business Use Case: This table is used to track and analyze sales transactions, including the products sold, customers, and sales amounts.

TRANSACTIONNUMBER: NVARCHAR(22): Unique identifier for each sales transaction 
PLANTCODE: NVARCHAR(10): Code representing the plant or location where the sale took place 
MATERIALCODE: NVARCHAR(25): Code representing the material or product sold 
CUSTOMERCODE: NVARCHAR(64): Code representing the customer who made the purchase 
STORECODE: NVARCHAR(2): Code representing the store where the sale took place 
SPENDINGDATE: NVARCHAR(10): Date when the sale was made 
ISMEMBER: NVARCHAR(1): Indicator whether the customer is a member or not
QTY: NUMERIC(13,3): Quantity of the material sold 
GROSSALES: NUMERIC(15,2): Total gross sales amount 
SPENDING: NUMERIC(15,2): Total spending amount
DISCOUNT: NUMERIC(15,2): Discount amount applied to the sale

Example data:  [{'TRANSACTIONNUMBER': '00202403100910267149', 'PLANTCODE': '64KA', 'MATERIALCODE': '000000000000228365', 'CUSTOMERCODE': '88B5AA46E74BA73B4FCB98252AAE2E73D2F795EE3CCF256905ED65D45145E915', 'STORECODE': 64, 'SPENDINGDATE': '2024-03-10', 'ISMEMBER': 'Y', 'QTY': 1.0, 'GROSSSALES': 299.0, 'SPENDING': 299.0, 'DISCOUNT': 0.0}]
"""

table2 = """
Table2: SALESTENDER_TRANSACTION 

Table Description: This table stores information about the payment tenders used in sales transactions, including the payment type, card details, and spending amounts. 
Table Type: Fact table 
Business Use Case: This table is used to track and analyze payment tenders used in sales transactions, including the types of payment methods, card networks, and spending patterns. 

TRANSACTIONNUMBER: NVARCHAR(22): Unique identifier for each sales tender transaction  
PAYTYPE: NVARCHAR(30): Code representing the payment type
PAYTYPE_DESC: NVARCHAR(22): Description of the payment type 
PAYCARDNETWORK: NVARCHAR(30): Network of the payment card
CARDTYPE:NVARCHAR(30): Type of payment card
PAYCARDISSUERS: NVARCHAR(30): Issuer of the payment card
CARDCLASS: NVARCHAR(30): Classification of the payment card
CARDNAME: NVARCHAR(120): Name of the payment card
ISPLATINUM: NVARCHAR(1): Indicator whether the payment card is a platinum card or not
SPENDING: NUMERIC(15,2): Total spending amount for this payment tender

Example data:  [{'TRANSACTIONNUMBER': '00202403100910174156', 'PAYTYPE': 'ZT07', 'PAYTYPE_DESC': 'Vendor coupon', 'PAYCARDNETWORK': nan, 'CARDTYPE': nan, 'PAYCARDISSUERS': nan, 'CARDCLASS': nan, 'CARDNAME': nan, 'ISPLATINUM': 'Y', 'TENDERSPENDING': -400.5}]
"""

table3 = """
Table3: PLANTCODE

Table Description: This table stores information about the plants or locations where sales transactions take place, including the unique plant code and plant name. 
Table Type: Dimension table 
Business Use Case: This table is used to provide context about the plants or locations where sales transactions occur, allowing for analysis and reporting by plant or location. 

PLANTCODE: NVARCHAR(10): Unique code identifying a plant or location 
PLANT_NAME: NVARCHAR(50): Name of the plant or location

Example data:  [{'PLANTCODE': '13KC', 'PLANT_NAME': 'III'}]
"""

table4 = """
Table4: PRODUCT_DIM

Table Description: This table stores information about the products sold, including the unique product code and hierarchical category descriptions. 
Table Type: Dimension table 
Business Use Case: This table is used to provide context about the products sold, allowing for analysis and reporting by product category, subcategory, and other product attributes. 

PRODUCTCODE: NVARCHAR(25): Unique code identifying a product 
CATEGORY_MCH1_DESC: NVARCHAR(100): Description of the first-level category for the product 
CATEGORY_MCH2_DESC: NVARCHAR(100): Description of the second-level category for the product 
CATEGORY_MCH3_DESC: NVARCHAR(100): Description of the third-level category for the product 
CATEGORY_MCH4_DESC: NVARCHAR(100): Description of the fourth-level category for the product

Example data:  [{'PRODUCTCODE': 103618, 'CATEGORY_MCH1DESC': 'ELECTRONICS', 'CATEGORY_MCH2DESC': 'IT', 'CATEGORY_MCH3DESC': 'GADGET & LIFESTLYE', 'CATEGORY_MCH4DESC': 'GP-GADGET & LIFESTLY'}]
"""

table5 = """
Table5: MCARD_DIM 

Table Description: This table stores information about the member cards, including the unique card code, member ID, application date, and card type description. 
Table Type: Dimension table 
Business Use Case: This table is used to provide context about the member cards, allowing for analysis and reporting by card type, member demographics, and card application trends. 

MCARDCODE: NVARCHAR(64): Unique code identifying a member's card 
MEMBERID: NVARCHAR(60): Unique identifier for the member who owns the card 
CARD_APPLYDATE: NVARCHAR(10): Date when the member applied for the card 
CARD_TYPEDESC: NVARCHAR(40): Description of the type of card

Example data:  [{'MCARDCODE': '51D7D55CC7C449250D020C7CFA5717189E4E960FF7A34202EB0B20942AC58984', 'MEMBERID': '1000000241', 'CARD_APPLYDATE': '2003-07-03', 'CARD_TYPEDESC': 'M PLATINUM CARD'}]
"""

table6 = """
Table6: DEMOGRAPHICS_DIM

Table Description: This table stores demographic information about members, including gender, segment profile, age generation, and age tier. 
Table Type: Dimension table 
Business Use Case: This table is used to provide context about member demographics, allowing for analysis and reporting by demographic segments, age ranges, and other demographic attributes. 

MEMBERID: NVARCHAR(60): Unique identifier for the member 
DEMO_GENDER: NVARCHAR(10): Gender of the member 
DEMO_SEGMENTPROFILE: NVARCHAR(8): Demographic segment profile of the member
DEMO_AGEGENERATION: NVARCHAR(20): Age generation of the member
DEMO_AGETIER: NVARCHAR(20): Age tier of the member

Example data:  [{'MEMBERID': 1004225744.0, 'DEMO_GENDER': 1, 'DEMO_SEGMENTPROFILE': 'SingleWorking', 'DEMO_AGEGENERATION': 'Gen-Y', 'DEMO_AGETIER': '36-45'}]
"""

table7 = """
Table7: BRANCH_DIM

Table Description: This table stores information about the branches, including the unique branch company code, and branch names in both Thai and English languages. 
Table Type: Dimension table 
Business Use Case: This table is used to provide context about the branches, allowing for analysis and reporting by branch location, branch type, and other branch attributes. 

BRANCH_COMPANYCODE: NVARCHAR(2): Unique code identifying a branch company 
BRANCH_NAMETH: NVARCHAR(30): Name of the branch in Thai language 
BRANCH_NAMEEN: NVARCHAR(30): Name of the branch in English language

Example data:  [{'BRANCH_COMPANYCODE': 13, 'BRANCH_NAMETH': 'บริษัท เอ', 'BRANCH_NAMEEN': 'Company A'}]
"""

In [ ]:
table_relationship = f"""
Table1:SALES_TRANSACTION.TRANSACTIONNUMBER [PK] to Table2: SALESTENDER_TRANSACTION.TRANSACTIONNUMBER [PK]
Table1:SALES_TRANSACTION.PLANTCODE [FK] to Table3: PLANTCODE.PLANTCODE [PK]
Table1:SALES_TRANSACTION.MATERIALCODE [FK] to Table4: PRODUCT_DIM.PRODUCTCODE [PK]
Table1:SALES_TRANSACTION.CUSTOMERCODE [FK] to Table5: MCARD_DIM.MCARDCODE [PK]
Table1:SALES_TRANSACTION.STORECODE [FK] to Table7: BRANCH_COMPANYCODE [PK]
Table5: MCARD_DIM.MEMBERID [FK] to Table6: DEMOGRAPHICS_DIM.MEMBERID [PK]
"""

In [ ]:
tables = [table1, table2, table3, table4, table5, table6, table7]


listA = tables

all_groups = []
for r in range(1, len(listA) + 1):
    groups = list(itertools.combinations(listA, r))
    all_groups.extend(groups)

# Convert groups from tuples to lists (if needed)
all_groups = [list(group) for group in all_groups]

all_str_group = []
for group in all_groups:
    table_desc = ''
    
    for i in group:
        table_desc = table_desc + '\n' + i
        
    all_str_group.append(table_desc)

In [ ]:
def create_question_and_sql(table_description, table_relationship):
    prompt = f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful AI assistant for data driven question to sql generation 
<|eot_id|><|start_header_id|>user<|end_header_id|>
Here is the data dictionary I have below

Table Details:
{table_description}

Table Relations:
Table1:SALES_TRANSACTION.TRANSACTIONNUMBER [PK] to Table2: SALESTENDER_TRANSACTION.TRANSACTIONNUMBER [PK]
Table1:SALES_TRANSACTION.PLANTCODE [FK] to Table3: PLANTCODE.PLANTCODE [PK]
Table1:SALES_TRANSACTION.MATERIALCODE [FK] to Table4: PRODUCT_DIM.PRODUCTCODE [PK]
Table1:SALES_TRANSACTION.CUSTOMERCODE [FK] to Table5: MCARD_DIM.MCARDCODE [PK]
Table1:SALES_TRANSACTION.STORECODE [FK] to Table7: BRANCH_COMPANYCODE [PK]
Table5: MCARD_DIM.MEMBERID [FK] to Table6: DEMOGRAPHICS_DIM.MEMBERID [PK]

I use netezza as my SQL generator.
Give me an example question and the SQL script. 
Using the tables that I have (ตอบเป็นภาษาไทย)
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
ต้องการ Pattern นี้ใช่ไหมครับ?

Question:
ขอดู CUSTOMERCODE ที่ซื้อสินค้ามากกว่า 2 ครั้ง ใน 1 วัน แยกตามวัน และ CUSTOMERCODE

SQL:
SELECT SPENDINGDATE, CUSTOMERCODE
     FROM SALES_TRANSACTION
     GROUP BY SPENDINGDATE, CUSTOMERCODE
     HAVING COUNT(TRANSACTIONNUMBER) > 2;
     
<|eot_id|><|start_header_id|>user<|end_header_id|>
Yes I want this pattern. Give me more 7 pairs with same pattern you gave me using all tables . (Question เขียนเป็นภาษาไทย)
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
Question:
    """
    return prompt

In [ ]:
all_prompts = []
for i in all_str_group:
    all_prompts.append(create_question_and_sql(i, table_relationship))

In [ ]:
#### WATSONX.AI ####
def send_to_watsonxai(model,
                    prompts
                    ):
    assert not any(map(lambda prompt: len(prompt) < 1, prompts)), "make sure none of the prompts in the inputs prompts are empty"
    output = []
    for prompt in prompts:
        o = model.generate_text(prompt)
        output.append(o)
    return output
    

In [ ]:
model_params = {
    GenParams.DECODING_METHOD: 'greedy',
    GenParams.TEMPERATURE: 0.33,
    GenParams.TOP_P: 0.67,
    GenParams.TOP_K: 82,
    GenParams.REPETITION_PENALTY: 1.0,
    GenParams.STOP_SEQUENCES:[']'],
    GenParams.MIN_NEW_TOKENS: 10,
    GenParams.MAX_NEW_TOKENS:4000
}
llm_model = Model(
    model_id=llama3,
    params=model_params,
    credentials=creds,
    project_id=project_id)

In [ ]:
datadriven_sql_pairs = []
for prompt in all_prompts:
    o = send_to_watsonxai(llm_model, [prompt])
    datadriven_sql_pairs.append(o[0])

In [ ]:
example_pairs_clean = ['Question:' + '\n' + i for i in simple_pairs]

In [ ]:
df_question_answer = pd.DataFrame()
df_question_answer['tableDescription'] = all_str_group
df_question_answer['questionExample'] = example_pairs_clean
df_question_answer.to_csv('table_example.csv', index=False)

In [ ]:
df = pd.read_csv('table_example.csv')
tobe_df = df.to_dict(orient = 'records')
df_list = []

for records in tobe_df:
    tableDescription = records['tableDescription']
    questionExample = records['questionExample']
    
    questions = questionExample.split('Question:')
    for question in questions:
        qu = question.split('SQL:')[0]
        qu = qu.replace('\n', '')
        if qu == '':
            continue
        else:
            # print(qu)
            # print(tableDescription)
            # print(questionExample)
            df_iterate = pd.DataFrame()
            df_iterate['ask_question'] = [qu]
            df_iterate['tableDescription'] = [tableDescription]
            df_iterate['questionExample'] = [questionExample]
            df_list.append(df_iterate)

df_final = pd.concat(df_list)
df_final.to_csv('tmg_search_tableDesc_Example.csv', index=False)

# Data driven question grooming

In [ ]:
# Abstract to Data driven
def create_data_driven_prompt(table_description, table_relationship):
    template = f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant for data driven company question guideline <|eot_id|><|start_header_id|>user<|end_header_id|>

Here is the data dictionary I have below

Table Details:
{table_description}

Table Relations:
Table1:SALES_TRANSACTION.TRANSACTIONNUMBER [PK] to Table2: SALESTENDER_TRANSACTION.TRANSACTIONNUMBER [PK]
Table1:SALES_TRANSACTION.PLANTCODE [FK] to Table3: PLANTCODE.PLANTCODE [PK]
Table1:SALES_TRANSACTION.MATERIALCODE [FK] to Table4: PRODUCT_DIM.PRODUCTCODE [PK]
Table1:SALES_TRANSACTION.CUSTOMERCODE [FK] to Table5: MCARD_DIM.MCARDCODE [PK]
Table1:SALES_TRANSACTION.STORECODE [FK] to Table7: BRANCH_COMPANYCODE [PK]
Table5: MCARD_DIM.MEMBERID [FK] to Table6: DEMOGRAPHICS_DIM.MEMBERID [PK]

Give me the example of abstract business question to data driven question.
5 Data driven questions in python list of string format per 1 abstract business data driven questions. (ตอบเป็นภาษาไทย)
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
ABSTRACT: 
"เราจะปรับปรุงความพึงพอใจของลูกค้าได้อย่างไร?"
DATA DRIVEN:
["อะไรคือปัจจัยหลักที่ส่งผลต่อความพึงพอใจของลูกค้าโดยพิจารณาจากข้อมูลการสำรวจล่าสุด?", "ผลิตภัณฑ์หรือบริการใดของเราที่ได้รับคะแนนความพึงพอใจต่ำที่สุดในช่วงไตรมาสที่ผ่านมา", "ความสัมพันธ์ระหว่างเวลาตอบสนองในการบริการลูกค้าและคะแนนความพึงพอใจของลูกค้าในช่วงหกเดือนที่ผ่านมาคืออะไร?", "ความพึงพอใจของลูกค้าแตกต่างกันไปในแต่ละกลุ่มประชากร (อายุ สถานที่ ประวัติการซื้อ) ตามความคิดเห็นล่าสุดอย่างไร?", "แนวโน้มคะแนนความพึงพอใจของลูกค้าในปีที่ผ่านมาคืออะไร และเหตุการณ์หรือการเปลี่ยนแปลงใดบ้างที่สอดคล้องกับการเปลี่ยนแปลงที่สำคัญ?"]

<|eot_id|><|start_header_id|>user<|end_header_id|>
OK your pattern is correct keep it this pattern. 
But please use the information from Table Details.
Create the ABSTRACT question and DATA DRIVEN question pairs. To make the data analyst easily create sql query from the tables we have, give the column name in the DATA DRIVEN question too.
Give me more 6 pairs, use above conditions. (ตอบเป็นภาษาไทย)

<|eot_id|><|start_header_id|>assistant<|end_header_id|>
1.
ABSTRACT:
"""
    return template

In [ ]:
all_prompts_question = []
for i in all_str_group:
    all_prompts_question.append(create_data_driven_prompt(i, table_relationship))

In [ ]:
question_grooming_pairs = []
for prompt in all_prompts_question:
    o = send_to_watsonxai(llm_model, [prompt])
    question_grooming_pairs.append(o[0])

In [ ]:
df_question_guide = pd.DataFrame()
df_question_guide['tableDescription'] = all_str_group
df_question_guide['exampleAbstracttoData'] = simple_pairs_question
df_question_guide.to_csv('tmg_search_question_conversion.csv', index = False)

In [ ]:
simple_pairs_question_clean = ['1.'+'\n'+'ABSTRACT:'+'\n'+i for i in simple_pairs_question]
simple_pairs_question_clean = [i.replace(' nào','า') for i in simple_pairs_question_clean]

simple_pairs_question_clean_number = []
for question in simple_pairs_question_clean:
    question = question.replace('1.', '')
    question = question.replace('2.', '')
    question = question.replace('3.', '')
    question = question.replace('4.', '')
    question = question.replace('5.', '')
    question = question.replace('6.', '')
    simple_pairs_question_clean_number.append(question)
    

In [ ]:
df_question_guide['exampleAbstracttoData_clean']=simple_pairs_question_clean_number
df_question_guide.to_csv('tmg_search_question_conversion.csv', index = False)
tobe_df = df_question_guide.to_dict(orient = 'records')
df_list = []

for records in tobe_df:
    tableDescription = records['tableDescription']
    questionExample = records['exampleAbstracttoData_clean']
    
    abstract_questions = questionExample.split('ABSTRACT:')
    for ab_question in abstract_questions:
        ab_qu = ab_question.split('DATA DRIVEN:')[0]
        ab_qu = ab_qu.replace('\n', '')
        if ab_qu == '':
            continue
        else:
            # print(qu)
            # print(tableDescription)
            # print(questionExample)
            df_iterate = pd.DataFrame()
            df_iterate['abstract_question'] = [ab_qu]
            df_iterate['tableDescription'] = [tableDescription]
            df_iterate['exampleAbstracttoData'] = [questionExample]
            df_list.append(df_iterate)

df_final = pd.concat(df_list)
df_final.to_csv('tmg_search_AbstractToData_Example.csv', index=False)

# VECTOR embedding and  DB creation

In [ ]:
from sentence_transformers import SentenceTransformer, models

def get_model(model_name='airesearch/wangchanberta-base-att-spm-uncased', max_seq_length=768, condition=True):
    if condition:
        # model_name = 'airesearch/wangchanberta-base-att-spm-uncased'
        # model_name = "hkunlp/instructor-large"
        word_embedding_model = models.Transformer(model_name, max_seq_length=max_seq_length)
        pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),pooling_mode='cls') # We use a [CLS] token as representation
        model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
    return model

In [ ]:

index = 'tmg_sql_query_prompt_body'

tmg_sql_query_prompt_body_dictionary = {
  "mappings": {
    "properties": {
        "ask_question": {"type": "text"},
        "question_example": {"type": "text"},
        "table_description": {"type":"text"},
        "embedding": {
            "type": "dense_vector",
            "dims": 768,
            "index": True,
            "similarity": "cosine"
        }
    }
  }
}

es.indices.create(index='tmg_sql_query_prompt_body', body= tmg_sql_query_prompt_body_dictionary)

In [ ]:
embeddings_model = get_model(model_name='kornwtp/simcse-model-phayathaibert', max_seq_length=768)
list_of_question=df_final['ask_question'].to_list()
list_of_table_description=df_final['tableDescription'].to_list()
list_of_example=df_final['questionExample'].to_list()

# VECTOR EMBEDDING
embeds = [list(embed) for embed in embeddings_model.encode(list_of_question)]
# VECTOR EMBEDDING

for table_description, question_example, ask_question, embedding in zip(list_of_table_description, list_of_example, list_of_question, embeds):
    table_dictionary_doc = {"table_description": table_description,
                        "question_example": question_example,
                        "ask_question": ask_question,
                        "embedding": embedding
                       }
    es.index(index=index, body=table_dictionary_doc)

In [ ]:
# 'abstract_question', 'tableDescription', 'exampleAbstracttoData'
index = 'tmg_abstract_to_data_prompt_body'

tmg_abstract_to_data_prompt_body_dictionary = {
  "mappings": {
    "properties": {
        "abstract_question": {"type": "text"},
        "abstract_to_data_example": {"type": "text"},
        "table_description": {"type":"text"},
        "embedding": {
            "type": "dense_vector",
            "dims": 768,
            "index": True,
            "similarity": "cosine"
        }
    }
  }
}

es.indices.create(index='tmg_abstract_to_data_prompt_body', body= tmg_abstract_to_data_prompt_body_dictionary)

In [ ]:
list_of_question=df_final['abstract_question'].to_list()
list_of_table_description=df_final['tableDescription'].to_list()
list_of_example=df_final['exampleAbstracttoData'].to_list()

# VECTOR EMBEDDING
embeds = [list(embed) for embed in embeddings_model.encode(list_of_question)]
# VECTOR EMBEDDING

for table_description, question_example, ask_question, embedding in zip(list_of_table_description, list_of_example, list_of_question, embeds):
    table_dictionary_doc = {"table_description": table_description,
                        "abstract_to_data_example": question_example,
                        "abstract_question": ask_question,
                        "embedding": embedding
                       }
    es.index(index=index, body=table_dictionary_doc)

In [ ]:
# TEST

def search_elastic(keyword_to_search, index_name, search_field):
    query = {
                "query": {
                    "match": {
                        f"{search_field}": keyword_to_search
                    }
                }
            }
    response = es.search(index=index_name, body=query)
    return response